In [1]:
import logging

import cv2
import matplotlib.pyplot as plt
import numpy as np

from methods import canny, hough_circle, show_img

config = {
    "./img/coins5.jpg": {  # 多硬币交叠,调试完成
        "high_threshold": 100,
        "low_threshold": 50,
        "min_r": 30,
        "max_r": 60,
        "min_voting": 8,
        "min_center_distance": 25,
        "debugging": False,
    },
}

原生python

In [2]:
%%timeit
%%capture
img_path = "./img/coins5.jpg"
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
circles = hough_circle(img, **config[img_path])

3.51 s ± 34.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


opencv c++实现

In [3]:
%%timeit 
%%capture 
# opencv例子
img = cv2.imread("./img/coins5.jpg", cv2.IMREAD_GRAYSCALE)
circles = cv2.HoughCircles(
    img,
    cv2.HOUGH_GRADIENT,
    dp=1,  # 图像分辨率和累加器分辨率比
    minDist=50,  # 被检测的圆心之间的最小距离
    param1=130,  # 传递给canny中的较高的阈值,低阈值是高阈值的一半
    param2=70,  # 检测阶段圆心的投票阈值，越小，错误的圆越多
    minRadius=30,  # 最小圆半径
    maxRadius=100,  # 最大圆半径
)
circles = np.uint16(np.around(circles))
img_c = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
for i in circles[0, :]:
    cv2.circle(img_c, (i[0], i[1]), i[2], (0, 255, 0), 2)
    cv2.circle(img_c, (i[0], i[1]), 2, (0, 0, 255), 2)
plt.imshow(img_c, "gray"), plt.axis("off"), plt.show()

77.8 ms ± 898 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


numba编译

In [4]:
%%timeit 
%%capture 

from methods_numba import canny_numba, hough_circle_numba, show_img_numba

# 针对图片的精调参数
config = {
    "./img/coins5.jpg": {  # 多硬币交叠,调试完成
        "high_threshold": 100,
        "low_threshold": 50,
        "min_r": 30,
        "max_r": 60,
        "min_voting": 8,
        "min_center_distance": 25,
        "debugging": False,
    },
}

img_path = "./img/coins5.jpg"
# test hough
img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
image, circles = hough_circle_numba(img, **config[img_path])
print(circles)

3.52 s ± 20.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
